In [ ]:
import jax
from jax import lax, random, numpy as jnp
from jax.tree_util import tree_flatten, tree_unflatten

import flax
from flax import linen as nn

import sympy as sy
from sympy.core.rules import Transform
import numpy as np

import sys
sys.path.append("..")
sys.path.append("../../orient/")


from eql.eqlearner import EQL, EQLdiv
from eql.symbolic import get_symbolic_expr_div, get_symbolic_expr
from np_utils import flatten, unflatten


import optax
import scipy
from functools import partial
import matplotlib.pyplot as plt

In [24]:
funs = ['mul', 'cos' , 'sin', 'id', 'id', 'id', 'id']
e = EQLdiv(n_layers=1, functions=funs, features=1)
key = random.PRNGKey(0)

In [35]:
N = 1024
xdim = 2
x = (random.uniform(key, (N, xdim))-.5) * 2
#x = np.array([[1., 2.]]).T
#x = np.linspace(-1, 1, N)[:,None]
#y = x[:,0] + jnp.cos(x[:,1])
#y = (jnp.cos(x) + 1 - x**2)/(x-3)**3


y = np.sin(np.pi * x[:,0])/(x[:,1]**2 + 1)
#y = 1./3. * ((1.+x[:,1])*np.sin(np.pi*x[:,0]) + x[:,1]*x[:,2]*x[:,3])
#plt.scatter(x[:,0], x[:,1], c=y)

In [36]:
params = e.init({'params':key}, x, 1.0);

In [27]:
def mse_fn(params, threshold):
    pred, _ = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2)

def mse_b_fn(params, threshold):
    pred, b = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2), b

def mse_b_y_fn(params, threshold):
    pred, b = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2), b, pred


def get_mask_spec(thresh, params):
    flat, spec = tree_flatten(params)
    mask = [jnp.abs(f) > thresh for f in flat]
    return mask, spec

def apply_mask(mask, spec, params):
    flat, _ = tree_flatten(params)
    masked_params = tree_unflatten(spec, [f*m for f,m in zip(flat, mask)])
    return masked_params


def get_masked_mse(thresh, params):
    mask, spec = get_mask_spec(thresh, params)
    def masked_mse(params, threshold):
        masked_params = apply_mask(mask, spec, params)
        return mse_fn(masked_params, threshold)
    return jax.jit(masked_mse)
    

def l1_fn(params):
    return sum(
        jnp.abs(w).mean() for w in jax.tree_leaves(params["params"])
    )

def reg_fn(threshold, b):
    return (jnp.maximum(0, threshold - b)).sum()

def penalty_fn(y, B=10, supp=3):
    penalty_fn.key, _ = random.split(key)
    xr = (random.uniform(penalty_fn.key, (N, xdim))-.5) * supp
    return jnp.sum(jnp.maximum(y-B, 0)+jnp.maximum(-y-B, 0))
penalty_fn.key = key

In [28]:
def get_loss(lamba):
    def loss_fn(params, threshold):
        mse, b = mse_b_fn(params, threshold)
        return mse  + lamba * l1_fn(params) + reg_fn(threshold, b)
    return loss_fn

def get_loss_pen():
    def loss_fn(params, threshold):
        mse, b, y = mse_b_y_fn(params, threshold)
        return penalty_fn(y) + reg_fn(threshold, b)
    return loss_fn

def get_loss_grad(lamba=1e-3, is_penalty=False):
    if is_penalty:
        loss = get_loss_pen()
    else:
        loss = get_loss(lamba)
    return jax.jit(jax.value_and_grad(loss))

In [29]:
tx = optax.adam(learning_rate=1e-4)
opt_state = tx.init(params)

In [30]:
loss_grad_pen = get_loss_grad(is_penalty=True)
loss_grad_1 = get_loss_grad(0)
loss_grad_2 = get_loss_grad(1e-1)

In [31]:
def do_step(loss_grad, params, theta, opt_state):
    loss_val, grad = loss_grad(params, theta)
    updates, opt_state = tx.update(grad, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss_val

In [34]:
T1 = 10_000
Tpenalty = 500
for i in range(20_000):
    theta = 1./jnp.sqrt(i/1. + 1)
    if i < T1:
        lg = loss_grad_1
    elif i >= T1:
        lg = loss_grad_2
    params, opt_state, loss_val = do_step(lg, params, theta, opt_state)
    if i % 99 == 0:
        print(loss_val, theta)
        for j in range(100):
            params, opt_state, loss_val = do_step(loss_grad_pen, params, theta, opt_state)

1.0463288 1.0
0.8063853 0.1
0.7134607 0.07088812
0.62582123 0.057928447
0.54472995 0.050188564
0.47112125 0.044901326
0.4055975 0.040996004
0.34845302 0.037959483
0.29969972 0.035511043
0.25908375 0.033482477
0.22609517 0.031766046
0.1999807 0.030289127
0.17978574 0.029000739
0.16444081 0.02786391
0.15288499 0.026851078
0.14418502 0.02594123
0.13759723 0.02511802
0.13256624 0.02436851
0.12868685 0.023682324
0.1256597 0.023051023
0.12325783 0.022467656
0.12130536 0.02192645
0.1196658 0.021422561
0.11823578 0.020951888
0.116939545 0.020510932
0.115723 0.020096697
0.11454752 0.019706586
0.11338485 0.019338345
0.11221367 0.018990004
0.111017816 0.018659834
0.10978538 0.018346308
0.108508125 0.018048072
0.10718068 0.017763922
0.10579964 0.017492786
0.1043628 0.017233696
0.10286854 0.016985789
0.10131577 0.016748281
0.09970393 0.016520465
0.098033346 0.016301699
0.09630523 0.0160914
0.09452173 0.015889037
0.09268546 0.01569412
0.0907993 0.015506205
0.08886569 0.015324884
0.08688657 0.0151497

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f4125b97700>
Traceback (most recent call last):
  File "/usr/lib/python3.8/weakref.py", line 345, in remove
KeyboardInterrupt: 


KeyboardInterrupt: 

In [12]:
T = 0
for i in range(10_000):
    theta = 1./jnp.sqrt(T/1 + 1)
    loss_val, grads = loss_grad_1(params, theta)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    T +=1 
    if i % 99 == 0:
        print(loss_val, theta)

for i in range(10_000):
    theta = 1./jnp.sqrt(T/1 + 1)
    loss_val, grads = loss_grad_2(params, theta)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    T +=1
    if i % 99 == 0:
        print(loss_val, theta)
        print(l1_fn(params))

NameError: name 'loss_grad_1' is not defined

In [37]:
# thr = 1e-3
# loss_grad_masked = jax.jit(jax.value_and_grad(get_masked_mse(thr, params)))
# mask, spec = get_mask_spec(thr, params)

# for i in range(1000):
#     theta = 1./jnp.sqrt(T/1 + 1)
#     loss_val, grads = loss_grad_masked(params, theta)
#     updates, opt_state = tx.update(grads, opt_state)
#     params = optax.apply_updates(params, updates)
#     T +=1
#     if i % 99 == 0:
#         print(loss_val)

In [38]:
#symb = get_symbolic_expr_div(apply_mask(mask, spec, params), funs)[0]
#symb = get_symbolic_expr_div(params, funs)[0]
#symb

In [129]:
spec, fparam = flatten(params)
full_shape = fparam.shape
mask = jnp.abs(fparam) > 0.01
idxs = jnp.arange(fparam.shape[0])[mask]
count = sum(mask).item()

In [130]:
def red_loss_grad_fn(red_param):
    full_param = jnp.zeros(full_shape).at[idxs].set(red_param)
    full_param = unflatten(spec, full_param)

    #return mse_fn(full_param, 1e-4)
    loss, grad = loss_grad_1(full_param, 1e-4)
    _, grad = flatten(grad)
    return loss, np.array(grad)[idxs,]
    
#red_loss_grad = jax.jit(jax.value_and_grad(red_mse_fn))

In [131]:
x0, f, info = scipy.optimize.fmin_l_bfgs_b(
        red_loss_grad_fn,
        x0 = np.array(fparam[mask]),
        factr=1.,
        m=500,
        pgtol=1e-13,
        maxls=100)
#x0[np.abs(x0) < 1e-3] = 0.0

In [132]:
f

DeviceArray(1.5276278e-08, dtype=float32)

In [133]:
final_param = unflatten(spec, jnp.zeros(full_shape).at[idxs].set(x0))

In [134]:
symb = get_symbolic_expr_div(final_param, funs)[0]
#symb

In [135]:
def clean_expr(expr):
    # WARNING: might return 0/NaN/inf if expression only contains small numbers
    def prune(expr, thr=1e-5):
        return expr.replace(lambda x: x.is_Number and abs(x) < thr, lambda x: 0)
    
    def rounding(expr, dig=3):
        return expr.xreplace(Transform(lambda x: x.round(dig), lambda x: x.is_Number))
    
    # prune small numbers
    expr = prune(expr)
    # round number
    expr = rounding(expr)
    # expand
    expr = prune(sy.expand(expr), 1e-3)
    return sy.simplify(expr)

In [136]:
clean_expr(symb)

(0.03108*x0**2 - 0.006742*x0*x1 + 0.387*x0*sin(1.554*x0 - 0.026*x1) + 0.01311*x0 + 0.249*x1*x2*x3 + 0.007547*x1*sin(1.554*x0 - 0.026*x1) + 0.002551*sin(2.083*x0) + 0.002551*sin(1.025*x0 - 0.052*x1) - 0.03413*sin(1.554*x0 - 0.026*x1) + 0.264*sin(3.036*x0 + 0.04*sin(1.554*x0 - 0.026*x1)) - 0.22*sin(0.168*cos(0.529*x0 + 0.026*x1) - 0.554) + 0.812*cos(0.003*x1) + 0.047*cos(0.529*x0 + 0.026*x1) + 2.502*cos(3.108*x0 - 0.052*x1) - 2.293*cos(3.18*x0 + 0.053*x1 - 0.013*sin(1.554*x0 - 0.026*x1)) - 1.15)/(-0.004*x0 + 0.003*sin(1.554*x0 - 0.026*x1) - 0.16*sin(0.168*cos(0.529*x0 + 0.026*x1) - 0.554) + 0.168*cos(0.003*x1) - 0.03*cos(0.529*x0 + 0.026*x1) + 0.003*cos(3.18*x0 + 0.053*x1 - 0.013*sin(1.554*x0 - 0.026*x1)) + 0.547)

In [17]:
f

DeviceArray(3.995705e-10, dtype=float32)

In [18]:
symb


0.333338635331777*x1*x2*x3 + 55.461997142339*sin(3.14159385455771*x0 - 0.00273492131132952*x1 + 1.56751096248627) + 55.4619321409225*sin(3.14159387018332*x0 + 0.00327529226449791*x1 - 1.56807163357735)